# Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings("ignore")

# Loading Data

In [ ]:
fake= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Fake.csv")
true= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/True.csv")

In [ ]:
fake.drop(columns=["title","date","subject"],inplace=True)
true.drop(columns=["title","date","subject"],inplace=True)

In [ ]:
true['label'] = 0
fake['label'] = 1

In [ ]:
print(f"Fake News : {fake.shape}")
print(f"Real News : {true.shape}")

Fake News : (23481, 2)
Real News : (21417, 2)


In [ ]:
df = pd.concat([true,fake])

In [ ]:
df =df.sample(frac=1, random_state=2000).reset_index(drop=True)

In [ ]:
print(f"dataset shape : {df.shape}")

dataset shape : (44898, 2)


# Preprocessing

In [ ]:
df[df['text'].isna() | (df['text'].str.strip() == '')].sum()

,0
text,...
label,630


In [ ]:
df = df.drop(df[df['text'].isna() | (df['text'].str.strip() == '')].index)
df.shape

(44267, 2)

In [ ]:
news = df.drop_duplicates().copy()
news.shape

(38644, 2)

In [ ]:
def preprocessing(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

### Stop Words are important in context so we will keep it

In [ ]:
news["processed"]=news["text"].apply(preprocessing)
news.head()

,text,label,processed
0,We know Trump is a sexual predator. We know he...,1,we know trump is a sexual predator we know he ...
1,William McGurn Wall Street JournalLet s get t...,1,william mcgurn wall street journallet s get th...
2,WASHINGTON (Reuters) - Iraqi Prime Minister Ha...,0,washington reuters iraqi prime minister haider...
3,WASHINGTON (Reuters) - President Donald Trump ...,0,washington reuters president donald trump spok...
4,Hillary was Secretary of State an unimaginable...,1,hillary was secretary of state an unimaginable...


# Prepare Data

In [ ]:
texts = news['processed'].tolist()
labels = news['label'].tolist()

# Train_Test_Split

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=2000)

In [ ]:
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

# Tokenizer

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=150)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/30915 [00:00<?, ? examples/s]

Map:   0%|          | 0/7729 [00:00<?, ? examples/s]

# Transformer

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
import transformers
print("Transformers version:", transformers.__version__)

Transformers version: 4.56.0


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1,
    report_to=None,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_steps=50,
    fp16=True,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Model

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.000000,0.019714,0.998965,0.998585,0.999150,0.998867
1000,0.000000,0.018240,0.999094,0.998867,0.999150,0.999009
1500,0.000000,0.018593,0.999094,0.998867,0.999150,0.999009
2000,0.000000,0.018807,0.999094,0.998867,0.999150,0.999009
2500,0.000000,0.018971,0.999094,0.998867,0.999150,0.999009
3000,0.000000,0.019094,0.999094,0.998867,0.999150,0.999009
3500,0.000000,0.019183,0.999094,0.998867,0.999150,0.999009
4000,0.000000,0.019254,0.999094,0.998867,0.999150,0.999009
4500,0.000000,0.019302,0.999094,0.998867,0.999150,0.999009
5000,0.000000,0.021010,0.998965,0.998303,0.999434,0.998868


TrainOutput(global_step=5799, training_loss=0.0005110997599370522, metrics={'train_runtime': 811.5278, 'train_samples_per_second': 114.284, 'train_steps_per_second': 7.146, 'total_flos': 3599322916509000.0, 'train_loss': 0.0005110997599370522, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.018239976838231087,
 'eval_accuracy': 0.9990943200931557,
 'eval_precision': 0.9988674971687429,
 'eval_recall': 0.9991503823279524,
 'eval_f1': 0.9990089197224975,
 'eval_runtime': 8.9865,
 'eval_samples_per_second': 860.066,
 'eval_steps_per_second': 53.858,
 'epoch': 3.0}

# Save Model

In [ ]:
model.save_pretrained("./fake_news_model")
tokenizer.save_pretrained("./fake_news_model")

('./fake_news_model/tokenizer_config.json',
 './fake_news_model/special_tokens_map.json',
 './fake_news_model/vocab.txt',
 './fake_news_model/added_tokens.json')